## Camera calibration for RidgeSfM

Based on https://nikatsanka.github.io/camera-calibration-using-opencv-and-python.html

Using your mobile phone camera, create a video of the chessboard pattern.
The video should be taken in landscape mode.
The phone should move about to view the pattern from different angles; see for example calibrate.3gp.

![a](pattern-8x8.png)

The code below will find your camera's intrinsic parameters, and prepare your videos for RidgeSfM
Then run
```
python ridgesfm.py scenes=calibrate/
```

In [1]:
# Set calibration_file to the name of the chessboard video.
# List your RidgeSfM input files in video_files

#Video of the 8x8 chessboard (mp4, 3gp or any other format)
calibration_file='calibrate.mp4' 

#Video of a room, with the same zoom settings as for the calibration video.
video_files=['room.mp4']  

In [2]:
import torchvision.transforms.functional as TTF
import torch, PIL.Image, glob, os, shutil, yaml, numpy as np, cv2
from tqdm import tqdm

if os.path.exists('calibration_images'):
    shutil.rmtree('calibration_images/')
os.makedirs('calibration_images',exist_ok=False)
os.system(f'ffmpeg  -i {calibration_file} -vf "setpts=0.25*PTS" calibration_images/image-%07d.png')
images=glob.glob('calibration_images/image*png')
for x in images:
    img=PIL.Image.open(x)
    if img.size[0]/img.size[1]>=640/480:
        img=TTF.resize(img,480)
    else:
        img=TTF.resize(img,640)
    img=TTF.center_crop(img,(480,640))
    assert img.size==(640,480)
    img.save(x[:-3]+'png')
print('Extracted', len(images), 'frames for calibration')

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

for fname in images:  # Here, 10 can be changed to whatever number you like to choose
    img = cv2.imread(fname) # Capture frame-by-frame
    #print(images[im_i])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)   # Certainly, every loop objp is the same, in 3D.
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

print("Number of images used for calibration: ", len(objpoints))

# calibration
camera_intrinsics = torch.tensor(cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)[1]).float()

print('Camera intrinsics')
print(camera_intrinsics)

Extracted 77 frames for calibration
Number of images used for calibration:  77
Camera intrinsics
tensor([[667.0068,   0.0000, 324.9735],
        [  0.0000, 667.6848, 239.3123],
        [  0.0000,   0.0000,   1.0000]])


In [3]:
#Pre-process videos for RidgeSfM

for ctr, x in enumerate(video_files):
    print('Pre-process',x)
    d=f'{os.getcwd()}/{ctr:06d}_images'
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d,exist_ok=False)
    os.system(f'ffmpeg  -i {x} {d}/image-%07d.png')
    files=sorted(glob.glob(f'{d}/image*'))
    for y in tqdm(files):
        img=PIL.Image.open(y)
        if img.size[0]/img.size[1]>=640/480:
            img=TTF.resize(img,480)
        else:
            img=TTF.resize(img,640)
        img=TTF.center_crop(img,(480,640))
        assert img.size==(640,480)
        img.save(y)

    for frameskip in [1,3,10,30]:
        s={'color': files[::frameskip],
           'intrinsic': camera_intrinsics,
           'pose': torch.stack([torch.eye(4) for _ in files]),
           'H': 480,
           'W': 640,
           'crop': (0,0)
        }
        torch.save(s,f'{d}/frameskip={frameskip}-{ctr:06d}.pth')
        print(f'run: python ridgesfm.py scenes=calibrate/ scene.n={ctr} scene.frameskip={frameskip}')

Pre-process room.mp4


100%|██████████| 752/752 [00:52<00:00, 14.34it/s]

run: python ridgesfm.py scenes=calibrate/ scene.n=0 scene.frameskip=1
run: python ridgesfm.py scenes=calibrate/ scene.n=0 scene.frameskip=3
run: python ridgesfm.py scenes=calibrate/ scene.n=0 scene.frameskip=10
run: python ridgesfm.py scenes=calibrate/ scene.n=0 scene.frameskip=30
